In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [7]:
def extract_histogram_features_fixed_resize(image_path, resize_shape=(200, 200), gray_levels=8, grid_size=(10, 10)):
    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize to fixed size (e.g., 200x200)
    img = cv2.resize(img, resize_shape)

    # Calculate size of each patch
    patch_h = resize_shape[0] // grid_size[0]
    patch_w = resize_shape[1] // grid_size[1]

    features = []

    # Calculate gradients using Sobel
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = cv2.magnitude(grad_x, grad_y)

    # Bin sizes
    gray_bin_size = 256 // gray_levels
    grad_max = gradient_magnitude.max()
    grad_bin_size = grad_max / gray_levels if grad_max != 0 else 1

    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            y1, y2 = i * patch_h, (i + 1) * patch_h
            x1, x2 = j * patch_w, (j + 1) * patch_w

            patch = img[y1:y2, x1:x2]
            grad_patch = gradient_magnitude[y1:y2, x1:x2]

            # Histogram of intensity
            hist_intensity, _ = np.histogram(patch, bins=gray_levels, range=(0, 256))
            hist_intensity = hist_intensity.astype(np.float32) / (patch.size + 1e-6)

            # Histogram of gradients
            hist_grad, _ = np.histogram(grad_patch, bins=gray_levels, range=(0, grad_max))
            hist_grad = hist_grad.astype(np.float32) / (grad_patch.size + 1e-6)

            # Combine
            patch_features = np.concatenate([hist_intensity, hist_grad])
            features.append(patch_features)

    # Final 1600-dimensional vector
    return np.concatenate(features)


In [8]:
def load_dataset(dataset_path, labels_dict):
    X, y = [], []
    for label_name, label_idx in labels_dict.items():
        folder = os.path.join(dataset_path, label_name)
        for filename in os.listdir(folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(folder, filename)
                features = extract_histogram_features_fixed_resize(image_path)
                X.append(features)
                y.append(label_idx)
    return np.array(X), np.array(y)


labels_dict = {'Tumor': 0, 'Stone': 1, 'Normal': 2, 'Cyst': 3}
X, y = load_dataset('processed_images/', labels_dict)


In [13]:

# نرمال‌سازی ویژگی‌ها
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# تغییر شکل به (تعداد نمونه، 100 پنجره، 16 ویژگی)
X_seq = X_scaled.reshape((X_scaled.shape[0], 100, 16))

# تبدیل لیبل‌ها به one-hot
y_cat = to_categorical(y, num_classes=4)


In [14]:

model = Sequential()
model.add(LSTM(200, input_shape=(100, 16)))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\Ayandegan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.fit(X_seq, y_cat, epochs=50, batch_size=32)


Epoch 1/50
 19/389 ━━━━━━━━━━━━━━━━━━━━ 51s 140ms/step - accuracy: 0.4658 - loss: 1.2382

KeyboardInterrupt: 

In [ ]:
predictions = model.predict(X_seq)
predicted_classes = predictions.argmax(axis=1)

# Accuracy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y, predicted_classes))
print("Confusion Matrix:\n", confusion_matrix(y, predicted_classes))
print("Classification Report:\n", classification_report(y, predicted_classes, target_names=labels_dict.keys()))


389/389 ━━━━━━━━━━━━━━━━━━━━ 15s 38ms/step
Accuracy: 1.0
Confusion Matrix:
 [[2283    0    0    0]
 [   0 1377    0    0]
 [   0    0 5077    0]
 [   0    0    0 3709]]
Classification Report:
               precision    recall  f1-score   support

       Tumor       1.00      1.00      1.00      2283
       Stone       1.00      1.00      1.00      1377
      Normal       1.00      1.00      1.00      5077
        Cyst       1.00      1.00      1.00      3709

    accuracy                           1.00     12446
   macro avg       1.00      1.00      1.00     12446
weighted avg       1.00      1.00      1.00     12446

